# Running Marrmot M01 using the ewatercycle system

### 1. Copy an example case

On https://github.com/wknoben/MARRMoT/tree/master/BMI/Config there is a example config file.

In [1]:
from pathlib import Path
forcing_file = Path('./BMI_testcase_m01_BuffaloRiver_TN_USA.mat')

In [ ]:
from ewatercycle.parametersetdb.datafiles import SubversionCopier
copier = SubversionCopier('https://github.com/wknoben/MARRMoT/trunk/BMI/Config/BMI_testcase_m01_BuffaloRiver_TN_USA.mat')
copier.save(forcing_file)

### 2. Verify your configuration settings

If you are working on a curated environment like https://lab.ewatercycle.org/, the default configuration should cover 90% of all use cases. However, for the other 10% here is a brief explanation of the relevant configuration settings. 

The ewatercycle configuration file is typcially stored in `~/.config/.ewatercycle/ewatercycle.yaml` or `/etc/ewatercycle.yaml` and can be imported with `from ewatercycle import CFG`. Relevant settings for any Marrmot model are:


In [3]:
import yaml
from ewatercycle.config import CFG

In [4]:
with open('./ewatercycle.yaml', 'w') as f:
    yaml.dump({
        'container_engine': 'docker', # or singularity
        'marrmot.docker_image': 'ewatercycle/marrmot-grpc4bmi:2020.11',
        'output_dir': './', # the directory where the model work_dir will be created
    }, f)

In [5]:
CFG.load_from_file('./ewatercycle.yaml')

### 3. Setting uip the model

In [6]:
from ewatercycle.models import MarrmotM01

/home/verhoes/miniconda38/envs/ewatercycle/lib/python3.9/site-packages/esmvalcore/experimental/_warnings.py:18: UserWarning: 
  Thank you for trying out the new ESMValCore API.
  Note that this API is experimental and may be subject to change.
  More info: https://github.com/ESMValGroup/ESMValCore/issues/498
/home/verhoes/miniconda38/envs/ewatercycle/lib/python3.9/site-packages/esmvalcore/experimental/config/_config_validators.py:254: ESMValToolDeprecationWarning: `write_plots` will be removed in 2.4.0.
/home/verhoes/miniconda38/envs/ewatercycle/lib/python3.9/site-packages/esmvalcore/experimental/config/_config_validators.py:255: ESMValToolDeprecationWarning: `write_netcdf` will be removed in 2.4.0.
/home/verhoes/miniconda38/envs/ewatercycle/lib/python3.9/site-packages/esmvalcore/experimental/config/_validated_config.py:80: MissingConfigParameter: `drs` is not defined (https://docs.esmvaltool.org/projects/ESMValCore/en/latest/quickstart/configure.html)


In [7]:
model = MarrmotM01()

In [8]:
model.parameters

[('maximum_soil_moisture_storage', 1000.0),
 ('initial_soil_moisture_storage', 900.0),
 ('solver',
  Solver(name='createOdeApprox_IE', resnorm_tolerance=0.1, resnorm_maxiter=6.0))]

In [9]:
cfg_file, cfg_dir = model.setup(
    forcing=forcing_file,
    maximum_soil_moisture_storage=1234.0,
)
cfg_file, cfg_dir

(PosixPath('/home/verhoes/git/eWaterCycle/ewatercycle/examples/marrmot_20210419_135246/marrmot-m01_config.mat'),
 PosixPath('/home/verhoes/git/eWaterCycle/ewatercycle/examples/marrmot_20210419_135246'))

In [10]:
model.parameters

[('maximum_soil_moisture_storage', 1234.0),
 ('initial_soil_moisture_storage', 900.0),
 ('solver',
  Solver(name='createOdeApprox_IE', resnorm_tolerance=0.1, resnorm_maxiter=6.0)),
 ('forcing_file', PosixPath('BMI_testcase_m01_BuffaloRiver_TN_USA.mat'))]

In [11]:
model.initialize(str(cfg_file))

In [23]:
model.update()

In [13]:
model.output_var_names

('P',
 'T',
 'Ep',
 'S(t)',
 'par',
 'sol_resnorm_tolerance',
 'sol_resnorm_maxiter',
 'flux_out_Q',
 'flux_out_Ea',
 'wb')

In [24]:
model.get_value('flux_out_Q')

array([4.16036419e-09])

In [28]:
model.get_value_as_xarray('flux_out_Q')

<xarray.DataArray 'flux_out_Q' (latitude: 1, longitude: 1)>
array([[4.16036419e-09]])
Coordinates:
  * longitude  (longitude) float64 87.49
  * latitude   (latitude) float64 35.29
    time       object 1989-01-04 00:00:00
Attributes:
    units:    mm day

In [29]:
model.finalize()

In [30]:
del model.bmi